In [0]:
dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()


Out[16]: '/Users/blahblah199517@gmail.com/Data-Loading-and-Cleaning'

In [0]:
!pip install pyspellchecker
!pip install --upgrade pyspellchecker
!pip install nltk

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-1c54cef1-a76c-4ba5-9c81-b6b99d977a89/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-1c54cef1-a76c-4ba5-9c81-b6b99d977a89/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-1c54cef1-a76c-4ba5-9c81-b6b99d977a89/bin/python -m pip install --upgrade pip' command.


In [0]:
import urllib
from pyspark.sql.functions import *
from pyspark.sql.types import *
import urllib
from spellchecker import SpellChecker
import nltk

In [0]:
dbutils.fs.ls('/FileStore/tables')

Out[19]: [FileInfo(path='dbfs:/FileStore/tables/databricks_manoj_accessKeys.csv', name='databricks_manoj_accessKeys.csv', size=99, modificationTime=1681655861000),
 FileInfo(path='dbfs:/FileStore/tables/pilla_bacha_accessKeys.csv', name='pilla_bacha_accessKeys.csv', size=99, modificationTime=1681863825000)]

In [0]:
# access key file type if csv
file_type = "csv"
# first row of the file contains the headers
first_row_is_header = "true"
# define the delimiter as ","
delimiter = ","
# read the file into spark 
manoj_aws_keys_df = spark.read.format(file_type).option("header", first_row_is_header).option("sep", delimiter).load("/FileStore/tables/databricks_manoj_accessKeys.csv")
vaish_aws_keys_df = spark.read.format(file_type).option("header", first_row_is_header).option("sep", delimiter).load("/FileStore/tables/pilla_bacha_accessKeys.csv")

In [0]:
VAISH_ACCESS_KEY = vaish_aws_keys_df.select("Access key ID").collect()[0]['Access key ID']
VAISH_SECRET_KEY = vaish_aws_keys_df.select("Secret access key").collect()[0]['Secret access key']
VAISH_ENCODED_SECRET_KEY = urllib.parse.quote(string=VAISH_SECRET_KEY, safe="")

In [0]:
VAISH_AWS_S3_BUCKET = "yelp-cleaned"
VAISH_SOURCE_URL = "s3n://{0}:{1}@{2}".format(VAISH_ACCESS_KEY, VAISH_ENCODED_SECRET_KEY, VAISH_AWS_S3_BUCKET)
MOUNT_POINT = "/mnt/yelp-cleaned"

# Check if the mount point exists
if not any(mount.mountPoint == MOUNT_POINT for mount in dbutils.fs.mounts()):
    # Mount the S3 bucket to the specified mount point
    dbutils.fs.mount(
    VAISH_SOURCE_URL,
    mount_point=MOUNT_POINT,
    extra_configs={"<key>": "<value>"}
  )
    print("S3 bucket mounted successfully!")
else:
    print("S3 bucket already mounted to the specified mount point.")


S3 bucket already mounted to the specified mount point.


In [0]:
dbutils.fs.ls("/mnt/yelp-cleaned")

Out[23]: [FileInfo(path='dbfs:/mnt/yelp-cleaned/business/', name='business/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/yelp-cleaned/checkin/', name='checkin/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/yelp-cleaned/review/', name='review/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/yelp-cleaned/tip/', name='tip/', size=0, modificationTime=0)]

## Business

In [0]:
business_csv = "/mnt/yelp-cleaned/business/business_csv.csv"

business_schema = StructType([
    StructField("business_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("city", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("stars", IntegerType(), True),
    StructField("review_count", IntegerType(), True),
    StructField("categories", StringType(), True)
])

business_df = spark.read.csv(business_csv, header=True, inferSchema=True)


In [0]:
business_df.select('business_id').distinct().count()

Out[25]: 10546

In [0]:
top_b_id = "/mnt/yelp-cleaned/business/top_100_business_ids.csv"
top_100_b_ids = spark.read.csv(top_b_id, header=True, inferSchema=True)

In [0]:
top_100_b_ids = top_100_b_ids.withColumnRenamed('business_ids', 'business_id')

business_df = business_df.drop('categories')
business_df = business_df.drop_duplicates()

In [0]:
top_100_business_df = business_df.join(top_100_b_ids, on='business_id')

top_100_business_df.show()


## Reviews

In [0]:
dbutils.fs.ls('/mnt/yelp-cleaned/review')

Out[27]: [FileInfo(path='dbfs:/mnt/yelp-cleaned/review/review_final.csv', name='review_final.csv', size=85500733, modificationTime=1683697302000)]

In [0]:
reviews = "/mnt/yelp-cleaned/review/review_final.csv"
# reviews = "/mnt/yelp-cleaned/review/review_final.json"
review_schema = StructType([
    StructField("business_id", StringType(), True),
    StructField("stars", IntegerType(), True),
    StructField("useful", IntegerType(), True),
    StructField("date", StringType(), True),
    StructField("text", StringType(), True)
])

review_df = spark.read.option("header", True).option("delimiter", ",").option("nullValue", "").schema(review_schema).csv(reviews)


In [0]:
# We selected review for 100 most popular restaurants. This Query should reflect the same
review_df.select('business_id').distinct().count()

Out[29]: 100

In [0]:
# Number of null values in each column of review_df
review_df.select([count(when(col(c).isNull(), c)).alias(c) for c in review_df.columns]).show()

+-----------+-----+------+----+----+
|business_id|stars|useful|date|text|
+-----------+-----+------+----+----+
|          0|    0|     0|   0|   9|
+-----------+-----+------+----+----+



In [0]:
# drop all null values 
review_df = review_df.dropna()

In [0]:
review_df.filter(col('text').isNull()).show()

+-----------+-----+------+----+----+
|business_id|stars|useful|date|text|
+-----------+-----+------+----+----+
+-----------+-----+------+----+----+



### Text cleaning

In [0]:
def clean_text(text):
    return " ".join(text.strip().split()).lower()

clean_text_udf = udf(clean_text, StringType())
# review_df = review_df.withColumn("text", clean_text_udf(trim(lower("text"))))

In [0]:
# Correcting date dtype
review_df = review_df.withColumn("date", to_timestamp("date", 'yyyy-MM-dd HH:mm:ss'))
# if 2 or more "." are consequtively present, replace it with "."
# review_df = review_df.withColumn("text", regexp_replace("text", r"\.{2,}", "."))
# lower case everything
# review_df = review_df.withColumn("text", lower(col('text')))
# removing special characters
review_df = review_df.withColumn("text", regexp_replace("text", '[!?]+', "."))
# Remove any other special character except fullstop and numbers
# review_df = review_df.withColumn("text", regexp_replace("text", '[^\w\s.]', ""))
# remove eol characters
# review_df = review_df.withColumn("text_1", regexp_replace("text", '[\n\r]', ""))



In [0]:
review_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- stars: integer (nullable = true)
 |-- useful: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- text: string (nullable = true)



### Stop word removal

In [0]:
from pyspark.ml.feature import StopWordsRemover

# Define the stopwords to remove
default_stop_words = StopWordsRemover.loadDefaultStopWords("english")

# add custom stop words that you want to retain
custom_stop_words = ["not", "no", "nor", "can't", 'cannot', 'could', "couldn't", "didn't", "doesn't", "don't", "hadn't", "hasn't", "haven't", "isn't", "shouldn't", "wasn't", "weren't"]
 
# combine the default and custom stop words into a new list
stop_words = list(set(default_stop_words) - set(custom_stop_words))

# Create a regular expression pattern to match the stopwords
stopwords_pattern = r'\b(' + '|'.join(stop_words) + r')\b\s*'



In [0]:
# Remove the stopwords from the "text" column
# review_df = review_df.withColumn("text_2", regexp_replace(col("text_1"), stopwords_pattern, ""))

### Spell Checker

In [0]:
# Initialize PySpellChecker
spell = SpellChecker()
 
# Define a user-defined function (UDF) to correct the spelling
def correct_spelling(text):
    corrected_words = []
    if text is None:
        return ''
    words = text.split()
    for word in words:
        period = ""
        if "." in word:
            period = "."
        if spell.correction(word) is not None:
            corrected_words.append(spell.correction(word) + period)
        else:
            corrected_words.append(word + period)
    return ' '.join(corrected_words)

# Register the function with spark
correct_spelling_udf = udf(correct_spelling)



In [0]:
# apply the udf to the dataframe column
review_df = review_df.withColumn('text', correct_spelling_udf('text'))


### Lemmatization

In [0]:
# Download necessary nltk packages
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Define lemmatizer function
wnl = nltk.WordNetLemmatizer()
def lemmatize_text(text):
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    lemmatized_tokens = [wnl.lemmatize(token, pos=get_wordnet_pos(pos_tag)) for token, pos_tag in tagged]
    return ' '.join(lemmatized_tokens)
 

# Define helper function to get WordNet POS tag from NLTK POS tag
def get_wordnet_pos(nltk_tag):
    if nltk_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif nltk_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN  # default to noun
    
# Create UDF for lemmatizing text
lemmatize_text_udf = udf(lemmatize_text, StringType())
   

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
# Apply UDF to DataFrame
# review_df = review_df.withColumn('text_4', lemmatize_text_udf(review_df['text_3']))

In [0]:
# review_df.cache()
review_df.count()

Out[42]: 136560

## Checkin

In [0]:
checkin = "/mnt/yelp-cleaned/checkin/top_100_checkins.csv"
checkin_schema = StructType([
    StructField("business_id", StringType(), True),
    StructField("date", StringType(), True)
])
checkin_df = spark.read.csv(checkin, schema=checkin_schema, header=True)

In [0]:
# Sanity Check
checkin_df.select('business_id').distinct().count()

Out[44]: 100

In [0]:
checkin_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)



In [0]:
# Delimit the checkin dates into a single value column
checkin_df = checkin_df.withColumn("date", split(checkin_df['date'], ','))
checkin_df = checkin_df.withColumn("date", explode(checkin_df['date']))
checkin_df = checkin_df.withColumn("date", trim(checkin_df['date']))

In [0]:
checkin_df = checkin_df.withColumn('date', to_timestamp('date', "yyyy-MM-dd HH:mm:ss"))

In [0]:
# latest checkin date for each business.
latest_checkin_df = checkin_df.groupby('business_id').agg(max('date').alias('latest_checkin'))
latest_checkin_df.orderBy(desc('latest_checkin')).show()

+--------------------+-------------------+
|         business_id|     latest_checkin|
+--------------------+-------------------+
|Sv1MEZP-mMfp8SmE0...|2022-01-19 16:23:04|
|_78Lam9Jd0W6ZmnHT...|2022-01-19 16:00:47|
|TNTBObKmICMn13T0p...|2022-01-19 14:56:05|
|nYsNsyIppe_DTueBh...|2022-01-19 11:41:06|
|XuQ5ekbJ9eRUX06JT...|2022-01-19 10:10:41|
|uYMVRPYEqpmVXWN2x...|2022-01-19 02:02:33|
|j-qtdD55OLfSqfsWu...|2022-01-19 01:48:31|
|8pqdJjefYq-a9IBSJ...|2022-01-18 23:43:43|
|aUoMG97DMJG4nmwhT...|2022-01-18 22:37:30|
|l4IeGBHXV2E8S8COd...|2022-01-18 22:36:45|
|p_qSQwShIgQnNxGca...|2022-01-18 14:52:25|
|X0vPZIkbUj22afBQz...|2022-01-18 04:41:20|
|3Z4rhPFO6XWJRQipE...|2022-01-18 03:39:19|
|L5LLN0RafiV1Z9cdd...|2022-01-18 02:22:57|
|GVWcwqUD-ekCMQ0x2...|2022-01-18 01:05:58|
|SwBhaxfQPbyhsi0QH...|2022-01-17 23:54:26|
|vUrTGX_7HxqeoQ_6Q...|2022-01-17 23:42:34|
|pXRrRf8fDv6yU3xp1...|2022-01-17 23:36:50|
|cd-rz-piYMftZtbFU...|2022-01-17 22:54:04|
|KP5OncF2jhT7_J1ph...|2022-01-17 20:52:13|
+----------

## Tips

In [0]:
tips = "/mnt/yelp-cleaned/tip/tips_100_business.csv"
tip_df = spark.read.csv(tips, inferSchema=True, header=True)

In [0]:
# We selected tips for 100 most popular restaurants. This Query should reflect the same
tip_df.select('business_id').distinct().count()

Out[50]: 100

In [0]:
# Number of null values in each column of Tip_df
# tip_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in tip_df.columns]).show()

tip_df.select([count(when(col(c).isNull(), c)).alias(c) for c in tip_df.columns]).show()


+-----------+----+----------------+----+
|business_id|date|compliment_count|text|
+-----------+----+----------------+----+
|          0|   0|               0|  31|
+-----------+----+----------------+----+



In [0]:
# remove entries with duplicate values
tip_df = tip_df.drop_duplicates()
tip_df = tip_df.dropna()

In [0]:
# Correcting date dtype
tip_df = tip_df.withColumn("date", to_timestamp("date", 'yyyy-MM-dd HH:mm:ss'))
# lower case everything
tip_df = tip_df.withColumn("text", lower(col('text')))
# removing special characters
tip_df = tip_df.withColumn("text", regexp_replace("text", '[!?]+', "."))


### Stop word removal

In [0]:
# Remove the stopwords from the "text" column
# tip_df = tip_df.withColumn("text", regexp_replace(col("text"), stopwords_pattern, ""))

### Spell Checker

In [0]:
# apply the udf to the dataframe column
tip_df = tip_df.withColumn('corrected_text', correct_spelling_udf('text'))


### Lemmatization

In [0]:
# Apply UDF to DataFrame
# tip_df = tip_df.withColumn('lemmatized_text', lemmatize_text_udf(tip_df['corrected_text']))

In [0]:
# Drop "complement_count"
tip_df = tip_df.drop("compliment_count", "text")

In [0]:
tip_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- corrected_text: string (nullable = true)

